<details>
<summary style="font-weight: 500; font-size: 24px">Загрузка модулей</summary>
Для запуска ячейки нажми <b>SHIFT + ENTER</b>.
</details>

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  # выбор GPU

import sys
sys.path.append('./src')
sys.path.append('./..')

In [ ]:
from ipy_utils import (IPYInteractiveSegmentation, show_video,
                       IPYRadioButtons, IPYButton)
from sam2al import SAM2
from cvat import CVATLabels
from cvat_srv import CVATSRV
from utils import TimeIt

<details>
<summary style="font-weight: 500; font-size: 24px">Открытие видеофайла</summary>
Для загрузки видеофайла необходимо скопировать путь к видео и вставить полученный путь в значение пременной <b style="font-family: 'Courier New', monospace;">video_file</b> (<i>можно использовать как абсолютный путь, так и относительный</i>).
</details>

In [ ]:
video_file = './project/videos/14_fragment_60.avi'

with TimeIt('загрузку модели'):
    sam2 = SAM2('/home/user/models/sam2.1_hiera_large.pt')

with TimeIt('загрузку видео'):
    sam2.load_video(video_file,
                    prompts_file='./project/prompts',
                    offload_video_to_cpu=True,
                    offload_state_to_cpu=False,
                    async_loading_frames=False)

<details>
<summary style="font-weight: 500; font-size: 24px">Загрузка списка классовй</summary>
Выбери класс объекта, для которого будет создан промпт.
</details>

In [ ]:
labeles = CVATLabels('./project/labeles.json')
IPYRadioButtons(**labeles.get_ipy_radio_buttons_kwargs('Класс объекта:'),
                setter=sam2.set_label).show();

<details>
<summary style="font-weight: 500; font-size: 24px">Интерактивная сегментация</summary>
В верхнем правом углу окна находится панель позиционирования кадра. Кнопка  приводит масштаб кадра к значению по-умолчанию (<i>по сути, это кнопка сброса масштаба</i>). С помощью стрелок   осуществляется переход между состоянями окна (<i>увы, CTRL + Z для отката не работает</i>). Кнопка  двигагает окно интерактивной сегментации по изображению с сохранением масштаба (<i>очень удобно для разметки малых объектов при сочетании с масштабированием; кнопка работает как «кнопка с залипанием»</i>).
Кнопка  обозначает выделение определенной области и приближение выделенной области до размеров окна. Кнопка также работает по принципу «кнопки с залипанием» (<i>после выделения нужной области необходимо щелкнуть по значку еще раз, поскольку интерфейс заблокирован, пока кнопка нажата</i>). И, наконец, кнопка  скачивает изображение окна интеркативной разметки вместе со всем содержимым (<i>можно использовать, чтобы поделиться результатом промптирования или для документирования</i>).
<br>
<br>
    
Используйте аргументы метода `go_to()` модуля `sam2` для перехода между кадрами и объектами. Аргумент `frame` - номер кадра, в котором будете ставить промпты, `obj_id` - номер объекта (будущего сегмента). Чтобы создать новый объект, нужно присвоить агрументу `obj_id` значение `'new'`. Также можно проставлять номера напрямую (*очень удобоно при редактировании промптов*). На скорость и удобство работы влияет выбранный опорный кадр (**первый размеченный** для класса кадр). Важно помнить, что не всегда первый кадр видео отлично подходит в качестве опорного (*к тому же, SAM2 может найти объект во всех кадрах до и после опорного*). Поэтому рекомендуется сначала просмотреть все видео, определить наилучший кадр в качестве опорного для размечаемого класса и только потом приступать к разметке. При большом количестве кадров рекомендуется ставить промпты на нескольких примерно равноудаленных друг от друга кадрах.

Если качество промптирования (выделения объектов с помощью промптов) какого-либо объекта на каком-либо кадре не удовлетворительно, необходимо перейти на этот кадр, определив `obj_id` проблемного объекта, и ставить промпты для проблемного объекта до тех пор, пока качество промптов не станет удовлетворительным.

Для создания прототипов сегметнов (масок) доступны следующие виды промптов: `include`, `exclude`, `box` и `mask`, - и кнопка `clear` для удаления всех промптов из кадра.
Разметка с помощью точек `include` применяется для хорошо видимых объектов с четкой однозначной границей и высокой цветовой контрастностью. Обычно применяется для выделения масок классов `adapter`, `ulabeled`, `trees` и `sky`. Промпт-точка `exclude` – удаление ложных масок (*если грамотно использовать точки `include` и `exclude`, то возможно разметить весь видеофайл только с помощью точек*). С помощью промпта `box` можно рамкой выделить область, в которой будет производится автоматическая установка маски.
Тип промпта `mask` позволяет в ручном режиме выделять маску; следует использовать для трудно различимых объектов, таких как `field` / `glade`, `glade` / `windrow` и т.д.

Для начала рекомендуется разметить классы `adapter`, `ulabeled`, `trees`, `transport` и `sky` с помощью промптов-точек, а затем остальные. Если модель SAM2 хорошо справляется с использованием промптов-точек для всех классов, тогда просто отлично, а если нет - пробуй использовать другие типы промтов для оставшихся кадров. Старайся использовать как можно меньше промптов (*это касается количества как самих промптов, так и их типов*). Если объект все равно плохо выделяется, (*например, валок*) зафиксируй этот момент как *issue* и работай дальше; в таком случае разметчики будут вручную дорабатывать.
</details>

In [ ]:
sam2.go_to(frame=411, obj_id='new')
ipis = IPYInteractiveSegmentation(**sam2.init_ipis_kwargs())

<details>
<summary style="font-weight: 500; font-size: 24px">Группировка определённых классов</summary>
При необходимости группировки нескольких масок с разными id в один будущий сегмент следует запустить эту ячейку По умолчанию исключениями для объединения являются классы <b>people</b>, <b>transport</b>, <b>field</b>. Если группировка ошибочна, запустить ячейку сброса всех группировок.
</details>

In [ ]:
for obj_id in sam2.prompts.get_all_obj_ids():
    if sam2.prompts.get_label(obj_id) not in ['People', 'transport', 'field']:
        sam2.prompts.set_group(obj_id, 1)
sam2.prompts.make_auto_save()

<details>
<summary style="font-weight: 500; font-size: 24px">Сброс всех группировок</summary>
Запусти эту ячейку, если группировка классов ошибочна.
</details>

In [ ]:
sam2.prompts.df['group'] = 0

## Таблица подсказок

In [ ]:
df = sam2.prompts.df

# Комментируй ненужное:

# selected_df = df[df['track_id'] == 6]     # Отобразить все подсказки заданного трека
# selected_df = df[df['label'] == 'glade']  # Отобразить все подсказки заданного класса
# selected_df = df[df['frame'] == 10]       # Отобразить все подсказки заданного кадра
selected_df = df                            # Отобразить все подсказки

# selected_df = selected_df.head(50)  # Берём верхние 50 записей
selected_df = selected_df.tail(50)  # Берём нижние 50 записей

selected_df

## Сборка превью

In [ ]:
with TimeIt('формирование превью без подгонки контуров'):
    video = show_video(sam2.render_preview('DraftPreview.mp4',
                                           fit_segments=False), size=480)
video

In [ ]:
with TimeIt('формирование превью с подгонкой контуров'):
    video = show_video(sam2.render_preview('CleanPreview.mp4',
                                           fit_segments=True), size=480)
video

## Создание XML-файла

In [ ]:
from IPython.display import display, HTML
import base64

In [ ]:
# Разметка:

xml_file = sam2.save2cvat_xml()
with open(xml_file, "rb") as file:
    xml_encoded = base64.b64encode(file.read()).decode('utf-8')
HTML(f'<a download="annotation.xml" href="data:text/xml;base64,{xml_encoded}" target="_blank">annotation.xml</a>')

In [ ]:
# Видео:

video_name = os.path.basename(sam2.video_file)
with open(sam2.video_file, "rb") as file:
    video_encoded = base64.b64encode(file.read()).decode('utf-8')
HTML(f'<a download="{video_name}" href="data:application/avi;base64,{video_encoded}" target="_blank">{video_name}</a>')

<details>
<summary style="font-weight: 500; font-size: 24px">Экспорт в CVAT</summary>
После запуска необходимо авторизоватся (система запрсит логин и пароль от CVAT). После успешной авторизации видео и файл с аннотациями будут загружены в CVAT (будет создана задача в указанном проекте). Если в процессе предразметки были какие-либо неточности или недоработки, нужно перейти в CVAT и указать все ошибки для их последующего устранения (<i>как уже упоминалось выше, либо ты сам или другой разметчик потом будет исправлять все недочеты предразметки</i>).
</details>

In [ ]:
# Эту ячейку достаточно запустить один раз за сессию.

cvat_srv = CVATSRV('http://89.169.168.184:8080')
print('Соединение с', str(cvat_srv), 'установлено.\n')

proj_name = '2024_Krasnodarskiy_krai'
proj = cvat_srv[proj_name]

In [ ]:
with TimeIt('cоздание/обновление задачи в CVAT'):
    url = sam2.export2cvat(proj)
url